In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import pymc as pm
from scipy.optimize import minimize

import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

nifty_50_tickers = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'HUL.NS',
    'HDFC.NS', 'KOTAKBANK.NS', 'BHARTIARTL.NS', 'AXISBANK.NS', 'ITC.NS', 'LT.NS',
    'NTPC.NS', 'TITAN.NS', 'MARUTI.NS', 'SUNPHARMA.NS', 'WIPRO.NS', 'ULTRACEMCO.NS',
    'SBIN.NS', 'M&M.NS', 'ASIANPAINT.NS', 'BAJAJ-FINANCE.NS', 'BAJAJ-AUTO.NS', 'HCLTECH.NS',
    'TATAMOTORS.NS', 'POWERGRID.NS', 'INDUSINDBK.NS', 'GRASIM.NS', 'BRITANNIA.NS',
    'JSWSTEEL.NS', 'TECHM.NS', 'DIVISLAB.NS', 'HDFCLIFE.NS', 'UPL.NS', 'COALINDIA.NS',
    'SHREECEM.NS', 'ADANIPORTS.NS', 'CIPLA.NS', 'ONGC.NS', 'DRREDDY.NS', 'HEROMOTOCO.NS',
    'EICHERMOT.NS', 'ICICIGI.NS', 'TATACONSUM.NS', 'SBILIFE.NS', 'MCDOWELL-N.NS', 'INDIAMART.NS',
    'DABUR.NS', 'NESTLEIND.NS', 'TATAPOWER.NS', 'HAVELLS.NS', 'BIOCON.NS', 'TATAMETALI.NS'
]
def fetch_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    data = data.dropna()
    return data

start_date = '2021-01-01'
end_date = '2024-01-01'

data = fetch_data(nifty_50_tickers, start_date, end_date)

returns = data.pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Step 4: Mean-Variance Optimization (Minimize Risk)
def portfolio_performance(weights, mean_returns, cov_matrix):
    port_return = np.sum(weights * mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return port_return, port_volatility

def min_variance(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for _ in range(len(nifty_50_tickers)))

initial_guess = np.ones(len(nifty_50_tickers)) / len(nifty_50_tickers)
result = minimize(min_variance, initial_guess, args=(mean_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = result.x
stock_weights = list(zip(nifty_50_tickers, optimal_weights))
stock_weights.sort(key=lambda x: x[1], reverse=True)

top_5_stocks = stock_weights[:5]

# Print the top 5 stocks and their weights
print("Top 5 Stocks Based on Risk-Return Optimization:")
for ticker, weight in top_5_stocks:
    print(f"{ticker}: {weight:.4f}")

tickers = [ticker for ticker, _ in top_5_stocks]

start_date_train = '2021-01-01'
end_date_train = '2024-01-01'
start_date_test = '2024-01-01'
end_date_test = '2025-01-01'

train_data = fetch_data(tickers, start_date_train, end_date_train)
test_data = fetch_data(tickers, start_date_test, end_date_test)

# Check for missing data
print("NaN values in train_data:", train_data.isnull().sum())

[*********************100%***********************]  53 of 53 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['HDFC.NS', 'MCDOWELL-N.NS', 'HUL.NS', 'BAJAJ-FINANCE.NS']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
ERROR:yfinance:['TATAMETALI.NS']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2021-01-01 -> 2024-01-01)')
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/loca

Top 5 Stocks Based on Risk-Return Optimization:
RELIANCE.NS: 0.0189
TCS.NS: 0.0189
HDFCBANK.NS: 0.0189
INFY.NS: 0.0189
ICICIBANK.NS: 0.0189



[*********************100%***********************]  5 of 5 completed


NaN values in train_data: Ticker
HDFCBANK.NS     0
ICICIBANK.NS    0
INFY.NS         0
RELIANCE.NS     0
TCS.NS          0
dtype: int64


In [ ]:
def bayesian_mean_variance_optimization(data, sentiment_values):
    returns = data.pct_change().dropna()
    mu = np.array(returns.mean()) * 100  # Mean returns (annualized)
    sigma = np.array(returns.cov())  # Covariance matrix
    mu_adjusted = mu * (1 + np.array(sentiment_values))  # Scale each stock's mean return by sentiment

    risk_aversion_factor = 1 + 0.5 * np.array(sentiment_values)
    sigma_adjusted = sigma * risk_aversion_factor[:, None]
    sigma_adjusted += np.eye(len(mu)) * 1e-4
    with pm.Model() as model:
        prior_mu = pm.Normal('mu', mu=0, sigma=1, shape=data.shape[1])
        prior_L = pm.LKJCholeskyCov('L', n=data.shape[1], eta=1, sd_dist=pm.HalfCauchy.dist(beta=2))


        prior_L_matrix = prior_L[0] if isinstance(prior_L, tuple) else prior_L
        prior_sigma = pm.Deterministic('sigma', prior_L_matrix @ prior_L_matrix.T)
        obs = pm.MvNormal('obs', mu=prior_mu, cov=prior_sigma, observed=data)
        approx = pm.fit(method="advi", n=5000,progressbar=False)
        trace = approx.sample(500)

    mu_posterior = trace.posterior['mu'].mean(dim=("chain", "draw")).values
    sigma_posterior = np.mean(trace.posterior['sigma'].values, axis=(0, 1))
    def objective(weights):
        return -np.dot(weights, mu_posterior) + 0.5 * np.dot(weights.T, np.dot(sigma_posterior, weights))

    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for _ in range(len(mu)))

    initial_guess = np.ones(len(mu)) / len(mu)

    result = minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x

In [ ]:
def get_user_sentiment():
    print("Please provide sentiment for each stock in the portfolio (use values between -1 and 1, 1 for very bullish and -1 for very bearish):")
    sentiment = {}
    for ticker in tickers:
        sentiment[ticker] = float(input(f"Sentiment for {ticker} (between -1 and 1): "))
    return list(sentiment.values())

sentiment_values = get_user_sentiment()


Please provide sentiment for each stock in the portfolio (use values between -1 and 1, 1 for very bullish and -1 for very bearish):
Sentiment for RELIANCE.NS (between -1 and 1): 0.7
Sentiment for TCS.NS (between -1 and 1): 0.8
Sentiment for HDFCBANK.NS (between -1 and 1): 0.9
Sentiment for INFY.NS (between -1 and 1): 0.9
Sentiment for ICICIBANK.NS (between -1 and 1): 1


In [ ]:
initial_weights = bayesian_mean_variance_optimization(train_data, sentiment_values)
print("Optimized Portfolio Weights:", initial_weights)

Optimized Portfolio Weights: [0.26477517 0.28324697 0.18624031 0.19755817 0.06817938]


In [ ]:
def backtest(test_data, initial_weights, sentiment_values, initial_amount=100000):
    weights = initial_weights
    portfolio_value = initial_amount
    allocation = portfolio_value * weights
    initial_prices = test_data.iloc[0].values
    shares = allocation / initial_prices

    prev_month = test_data.index[0].month

    for i in range(1, len(test_data)):
        current_prices = test_data.iloc[i].values
        portfolio_value = np.sum(shares * current_prices)
        new_allocation = portfolio_value * weights

        current_month = test_data.index[i].month

        # Rebalance every 2 months
        if current_month != prev_month and (current_month - prev_month) % 2 == 0:
            print(f"Rebalancing on {test_data.index[i]}")

            # Re-optimize with user sentiment adjustment (no need to ask for sentiment again)
            weights = bayesian_mean_variance_optimization(test_data.iloc[:i], sentiment_values)

            # Recalculate portfolio allocation based on new weights
            new_allocation = portfolio_value * weights
            shares = new_allocation / current_prices
            prev_month = current_month

    return portfolio_value, weights


In [ ]:
final_portfolio_value, final_weights = backtest(test_data, initial_weights, sentiment_values)

# Output final results
print("Final Portfolio Value:", final_portfolio_value)
print("Net Profit:", final_portfolio_value - 100000)
print("Profit % :", (final_portfolio_value - 100000) / 100000 * 100)  # Percentage profit


Rebalancing on 2024-03-01 00:00:00
Rebalancing on 2024-05-02 00:00:00
Rebalancing on 2024-07-01 00:00:00
Rebalancing on 2024-09-02 00:00:00
Rebalancing on 2024-11-01 00:00:00
Final Portfolio Value: 117615.0643496916
Net Profit: 17615.064349691602
Profit % : 17.615064349691604
